<a href="https://colab.research.google.com/github/charlesfg/term.ooo-solver/blob/main/term_ooo_cheat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

# https://www.gabtoschi.com/letreco/
# https://term.ooo/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
words5char = set()
with open('/content/drive/MyDrive/CC/br-sem-acentos.txt') as f:
  for l in f:
    w = l.strip()
    if len(w) == 5:
      words5char.add(w.lower())

print(f"Number of words of 5 characters {len(words5char)}")

with open('/content/drive/MyDrive/CC/words5char.txt', 'w') as f:
  f.write("\n".join(words5char))


Number of words of 5 characters 5427


In [46]:
from google.colab import files
import json

with open("words5char.json", "w") as f:
  json.dump(list(words5char),f)

files.download("words5char.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
from collections import defaultdict

# character frequency
cf = defaultdict(int)
num_c = 0

for w in words5char:
  for c in w:
    cf[c] +=1
    num_c +=1




import operator
print(sorted(cf.items(), key=operator.itemgetter(1), reverse=True))


[('a', 4244), ('o', 2765), ('e', 2739), ('i', 2157), ('r', 2093), ('s', 1968), ('m', 1293), ('u', 1262), ('t', 1097), ('c', 1086), ('l', 1021), ('n', 868), ('d', 754), ('p', 691), ('g', 598), ('v', 591), ('b', 535), ('f', 530), ('h', 233), ('j', 205), ('z', 201), ('x', 162), ('q', 38), ('k', 4)]


In [48]:
words5char_freq = {}
for k,v in sorted(cf.items(), key=operator.itemgetter(1), reverse=True):
  print(k,"{0:.4f}".format(v/num_c))


a 0.1564
o 0.1019
e 0.1009
i 0.0795
r 0.0771
s 0.0725
m 0.0477
u 0.0465
t 0.0404
c 0.0400
l 0.0376
n 0.0320
d 0.0278
p 0.0255
g 0.0220
v 0.0218
b 0.0197
f 0.0195
h 0.0086
j 0.0076
z 0.0074
x 0.0060
q 0.0014
k 0.0001


In [140]:
import re
import operator
import random

# Model of 5 character words 

vowel_f = {
    "a" : 0.1564,
    "o" : 0.1019,
    "e" : 0.1009,
    "i" : 0.0795,
    "u" : 0.0465,
} 

consonant_f = {
    "r" : 0.0771,
    "s" : 0.0725,
    "m" : 0.0477,
    "t" : 0.0404,
    "c" : 0.0400,
    "l" : 0.0376,
    "n" : 0.0320,
    "d" : 0.0278,
    "p" : 0.0255,
    "g" : 0.0220,
    "v" : 0.0218,
    "b" : 0.0197,
    "f" : 0.0195,
    "h" : 0.0086,
    "j" : 0.0076,
    "z" : 0.0074,
    "x" : 0.0060,
    "q" : 0.0014,
    "k" : 0.0001,
}



"""
Given a set of test characteres it will test if they relates to the word_str
returning trues if this relation holds.

Ex.  check('ab',"colegio","in") ==> False
     check('ab',"colegio","out") ==> True
     check('ab',"colegio","biro") ==> TypeError
"""
def check(test_chars,word_str, relations):
  if relations == "in":
    for i in test_chars:
      if i not in word_str:
        return False
    return True
  if relations == "out":
    for i in test_chars:
      if i in word_str:
        return False
    return True
  raise TypeError("relations should be in/out")


"""
in_word       - characters that must exist in the word
out_of_word   - characters that can't exist in the word
word_regex    - regex describing the word (which characters and in 
      which position), ex. '^.a..s$' for 'patos'
"""
def print_candidates(in_word, out_of_word, word_regex, ret=False):
  candidates = []
  for w in words5char:
    if word_regex:
      if not re.match(word_regex, w):
        continue
    if not check(in_word, w, "in"):
      continue
    if not check(out_of_word, w, "out"):
      continue
    
    candidates.append(w)

  if ret:
    return candidates

  for i in candidates:
    print(i)


def print_candidates_trained(in_word, out_of_word, word_regex):
  iw_l = len(in_word)

  vc = vowel_f.copy()
  cc = consonant_f.copy()

  if iw_l == 5:
    print_candidates(in_word,out_of_word, word_regex)
    return

  for o in out_of_word+in_word:
    if o in vc:
      del vc[o]
      print("removing ",o)
    if o in cc:
      del cc[o]
      print("removing ",o)
 
  
  if iw_l == 0:
    tmp_inw = "".join([x[0] for x in sorted(vc.items(), key=operator.itemgetter(1), reverse=True)][0:3])
    tmp_inw += "".join([x[0] for x in sorted(cc.items(), key=operator.itemgetter(1), reverse=True)][0:2])
    print_candidates(tmp_inw, out_of_word, word_regex)    
    return 

  new_chars = ""
  candidates = None
  print("-",in_word)
  
  for i in range(5-iw_l):    
    char_source = cc
    # Just one vowel per try after the first try
    if i == 0:
      if vc:
        char_source = vc
    tmp_c = [x[0] for x in sorted(char_source.items(), key=operator.itemgetter(1), reverse=True)][0]
    print(" tmp_c:",tmp_c)
    new_chars += tmp_c
    del char_source[tmp_c]  
    print("-",in_word+new_chars)

  while not candidates:
    new_chars= new_chars[:-1]
    print("failed... retrying with ",new_chars)
    candidates = print_candidates(in_word+new_chars,out_of_word, word_regex, True)
  
  print(candidates)
    
 
  



In [147]:
print_candidates_trained("","","")
#menprint_candidates_trained("roe","sa","[^r][^o][^s][^e][^a]")
print_candidates_trained("rome","isa","[^ri][^ro][^es][^me][^ao]")
#print_candidates_trained("cuom","pseirta","c[^u][^p][^o]m")
#print_candidates("omit","sera","[^s][^e][^r][^a][^o]")

serao
rosea
removing  s
removing  a
removing  r
removing  o
removing  e
- roe
 tmp_c: i
- roei
 tmp_c: m
- roeim
failed... retrying with  i
['iremo', 'epiro', 'ibero', 'erico', 'creio', 'freio', 'piore', 'heroi', 'erijo', 'beiro', 'erodi', 'itero', 'ebrio']
removing  i
removing  s
removing  a
removing  r
removing  o
removing  m
removing  e
- rome
 tmp_c: u
- romeu
failed... retrying with  
['menor', 'temor']


In [111]:
import random
str_var = list("mitc")
random.shuffle(str_var)
print(''.join(str_var)[0:-1])

icm


In [70]:
import copy


# Model of 5 character words 

vowel_f = {
    "a" : 0.1564,
    "o" : 0.1019,
    "e" : 0.1009,
    "i" : 0.0795,
    "u" : 0.0465,
} 

consonant_f = {
    "r" : 0.0771,
    "s" : 0.0725,
    "m" : 0.0477,
    "t" : 0.0404,
    "c" : 0.0400,
    "l" : 0.0376,
    "n" : 0.0320,
    "d" : 0.0278,
    "p" : 0.0255,
    "g" : 0.0220,
    "v" : 0.0218,
    "b" : 0.0197,
    "f" : 0.0195,
    "h" : 0.0086,
    "j" : 0.0076,
    "z" : 0.0074,
    "x" : 0.0060,
    "q" : 0.0014,
    "k" : 0.0001,
}

p = 3
tmp_inw = "".join([x[0] for x in sorted(vowel_f.items(), key=operator.itemgetter(1), reverse=True)][0:p])
tmp_inw += "".join([x[0] for x in sorted(consonant_f.items(), key=operator.itemgetter(1), reverse=True)][0:2])
print(tmp_inw)

aoers


In [9]:

g_vow_freq = {
'a' : 0.1463,
'i' : 0.0618,
'u' : 0.0463,
'e' : 0.01257,
'o' : 0.01073,
}

g_cons_freq = {

'b' : 0.0104,
'c' : 0.0388,
'd' : 0.0499,

'f' : 0.0102,
'g' : 0.0130,
'h' : 0.0128,

'j' : 0.0040,
'k' : 0.0002,
'l' : 0.0278,
'm' : 0.0474,
'n' : 0.0505,

'p' : 0.0252,
'q' : 0.0120,
'r' : 0.0653,
's' : 0.0781,
't' : 0.0434,

'v' : 0.0167,
'w' : 0.0001,
'x' : 0.0021,
'y' : 0.0001,
'z' : 0.0047,
}

import operator
print(sorted(g_vow_freq.items(), key=operator.itemgetter(1), reverse=True))
print(sorted(g_cons_freq.items(), key=operator.itemgetter(1), reverse=True))



[('a', 0.1463), ('i', 0.0618), ('u', 0.0463), ('e', 0.01257), ('o', 0.01073)]
[('s', 0.0781), ('r', 0.0653), ('n', 0.0505), ('d', 0.0499), ('m', 0.0474), ('t', 0.0434), ('c', 0.0388), ('l', 0.0278), ('p', 0.0252), ('v', 0.0167), ('g', 0.013), ('h', 0.0128), ('q', 0.012), ('b', 0.0104), ('f', 0.0102), ('z', 0.0047), ('j', 0.004), ('x', 0.0021), ('k', 0.0002), ('w', 0.0001), ('y', 0.0001)]


In [21]:
#print_candidates("eorm","cguapis","[^rac]e[^egm]o[^ous]")
#print_candidates("aiusr","","")
print_candidates("reonm","duias","[^rd][^ur][^ie][^an][^so]")

menor
